In [1]:
import pandas as pd
import numpy as np
import re
import os
import utils 
import string

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [2]:
data_dir = "data/"

files = ["H-1B_Disclosure_Data_FY16.xlsx", 
            "H-1B_Disclosure_Data_FY15_Q4.xlsx", 
            "H-1B_FY14_Q4.xlsx", 
            "LCA_FY2013.xlsx"]



In [3]:
data = pd.read_excel(data_dir + files[0])

In [4]:
columns = list(data)
print(columns)
print(data.dtypes)


['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAIC_CODE', 'TOTAL_WORKERS', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'H-1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'ORIGINAL_CERT_DATE']
CASE_NUMBER                      object
CASE_STATUS                      object
CASE_SUBMITTED           datetime64[ns]
DECISION_DATE            datetime64[ns]
VISA_CLASS                       object
EMPLOYMENT_START_DATE    datetime64[ns

In [5]:
data.head()

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-12240-490687,CERTIFIED-WITHDRAWN,2012-08-27,2015-10-16,H-1B,2013-02-25,2016-02-24,UNIVERSITY OF MICHIGAN,"PIERPONT COMMONS, INTERNATIONAL CENTER",ANN ARBOR,MI,48109,UNITED STATES OF AMERICA,NaN,7347634081,NaN,",",NaN,NaN,POSTDOCTORAL RESEARCH FELLOW,10-1021.00,BIOCHEMISTS AND BIOPHYSICISTS,611310.0,1,NaN,36067.00,Year,OES,2012.0,NaN,41000.00,0.0,Year,N,N,ANN ARBOR,WASHTENAW,MI,48109,2012-08-31
1,I-200-13053-847481,CERTIFIED-WITHDRAWN,2013-02-25,2016-01-13,H-1B,2013-08-24,2016-08-23,"GOODMAN NETWORKS, INC.","6400 INTERNATIONAL PARKWAY, SUITE 1000",PLANO,TX,75093,UNITED STATES OF AMERICA,NaN,9724215173,NaN,"HULME, RANDALL",ADDISON,TX,CHIEF OPERATING OFFICER,11-1011,CHIEF EXECUTIVES,238210.0,1,NaN,242674.00,Year,OES,2012.0,OFLC ONLINE DATA CENTER,400000.00,0.0,Year,N,N,PLANO,COLLIN,TX,75093,2013-03-01
2,I-200-13088-054259,CERTIFIED-WITHDRAWN,2013-04-23,2015-12-10,H-1B,2013-04-29,2016-04-28,"PORTS AMERICA GROUP, INC.",525 WASHINGTON BOULEVARD,JERSEY CITY,NJ,07310,UNITED STATES OF AMERICA,NaN,7326353899,NaN,"HENNER, JANET",NEW YORK,NY,CHIEF PROCESS OFFICER,11-1011,CHIEF EXECUTIVES,488310.0,1,NaN,193066.00,Year,OES,2012.0,OFLC ONLINE DATA CENTER,264000.00,451100.0,Year,N,N,JERSEY CITY,HUDSON,NJ,07310,2013-04-29
3,I-200-13144-034110,CERTIFIED-WITHDRAWN,2013-05-24,2016-01-19,H-1B,2013-10-01,2016-09-30,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",1551 WEWATTA STREET,DENVER,CO,80202,UNITED STATES OF AMERICA,NaN,3037444739,NaN,"Shackelford, Andrew",PHOENIX,AZ,"REGIONAL PRESIDEN, AMERICAS",11-1011,CHIEF EXECUTIVES,3363.0,1,NaN,220314.00,Year,OES,2012.0,OFLC ONLINE DATA CENTER,220314.00,600000.0,Year,N,N,DENVER,DENVER,CO,80202,2013-05-29
4,I-200-13172-415116,WITHDRAWN,2013-06-26,2016-05-20,H-1B,2013-10-18,2016-10-17,PEABODY INVESTMENTS CORP.,701 MARKET STREET,ST. LOUIS,MO,63101,UNITED STATES OF AMERICA,NaN,3145883065,NaN,"BAILEY, MICHAEL",ADDISON,TX,PRESIDENT MONGOLIA AND INDIA,11-1011,CHIEF EXECUTIVES,213113.0,1,NaN,75.73,Hour,OES,2012.0,OFLC ONLINE DATA CENTER,171.63,0.0,Hour,N,N,ST. LOUIS,ST. LOUIS,MO,63101,NaT


In [6]:
print(len(data))

647852


In [7]:
print(len(data.CASE_NUMBER.unique()))

647849


In [8]:
count = data.groupby('CASE_NUMBER').count()

In [9]:
data.loc[data['CASE_NUMBER'].isin(count[count['CASE_STATUS'] > 1].index)]

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
28062,I-200-15061-542522,CERTIFIED-WITHDRAWN,2015-03-02,2016-01-14,H-1B,2015-09-01,2018-09-01,"INOVALON, INC.",4321 COLLINGTON ROAD,BOWIE,MD,20716,UNITED STATES OF AMERICA,NaN,3018094000,1877,"RUDINSKY, AGNES",BOWIE,MD,BUSINESS ANALYST,13-1111,MANAGEMENT ANALYSTS,541690.0,1,NaN,61797.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,61797.0,71797.0,Year,N,N,BOWIE,PRINCE GEORGE'S,MD,20716,2015-03-10
28063,I-200-15061-542522,CERTIFIED-WITHDRAWN,2015-03-02,2016-01-14,H-1B,2015-09-01,2018-09-01,"INOVALON, INC.",4321 COLLINGTON ROAD,BOWIE,MD,20716,UNITED STATES OF AMERICA,NaN,3018094000,1877,"RUDINSKY, AGNES",BOWIE,MD,SENIOR BUSINESS ANALYST,13-1111,MANAGEMENT ANALYSTS,541690.0,1,NaN,80413.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,80413.0,90413.0,Year,N,N,BOWIE,PRINCE GEORGE'S,MD,20716,2015-03-10
82847,I-200-15061-542522,WITHDRAWN,2015-03-02,2016-09-30,H-1B,2015-09-01,2018-09-01,"INOVALON, INC.",4321 COLLINGTON ROAD,BOWIE,MD,20716,UNITED STATES OF AMERICA,NaN,3018094000,1877,"RUDINSKY, AGNES",BOWIE,MD,BUSINESS ANALYST,15-1121,COMPUTER SYSTEMS ANALYSTS,541690.0,1,NaN,67621.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,72000.0,82000.0,Year,N,N,BOWIE,PRINCE GEORGE'S,MD,20716,NaT
199703,I-200-15070-889948,CERTIFIED-WITHDRAWN,2015-03-11,2016-02-18,H-1B,2015-09-01,2018-03-30,"UNIFY SOLUTIONS, INC.",4 WEST ROLLING CROSSROADS,CATONSVILLE,MD,21228,UNITED STATES OF AMERICA,NaN,4104555152,104,",",NaN,NaN,PROGRAMMER ANALYST,15-1131,COMPUTER PROGRAMMERS,541511.0,1,NaN,53040.0,Year,OES,2014.0,OFLC ONLINE DATA CENTER,60000.0,0.0,Year,Y,N,CATONSVILLE,BALTIMORE,MD,21228,2015-03-18
438985,I-200-15070-889948,CERTIFIED-WITHDRAWN,2015-03-23,2016-02-18,H-1B,2015-09-22,2017-12-30,"UNIFY SOLUTIONS, INC.",4 WEST ROLLING CROSSROADS,CATONSVILLE,MD,21228,UNITED STATES OF AMERICA,NaN,4104555152,104,",",NaN,NaN,SYSTEMS ADMINISTRATOR,15-1142,NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,541511.0,1,NaN,56950.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,57000.0,0.0,Year,Y,N,CATONSVILLE,BALTIMORE,MD,21228,2015-03-27


In [10]:
print(data.CASE_STATUS.unique())

['CERTIFIED-WITHDRAWN' 'WITHDRAWN' 'CERTIFIED' 'DENIED']


In [11]:
print(data.VISA_CLASS.unique())

['H-1B' 'H-1B1 Chile' 'H-1B1 Singapore' 'E-3 Australian'
 'Select Visa Classification']


In [12]:
data[data.VISA_CLASS == 'Select Visa Classification']

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
647849,I-200-16063-456774,WITHDRAWN,2016-03-03,2016-03-03,Select Visa Classification,NaT,NaT,TECHNOPLUS INC,1004 RIDGE HOLLOW TR,IRVING,TX,75063,UNITED STATES OF AMERICA,NaN,2142507565,NaN,",",NaN,NaN,NaN,NaN,NaN,541511.0,0,NaN,71032.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,0.0,0.0,NaN,NaN,NaN,COPPELL,DALLAS,TX,75019,NaT


In [13]:
data.VISA_CLASS = data.VISA_CLASS.replace('Select Visa Classification', 'H-1B')

In [14]:
data[data.VISA_CLASS == 'Select Visa Classification']

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE


In [15]:
data.CASE_SUBMITTED.isnull().sum()

0

In [16]:
data.DECISION_DATE.isnull().sum()

0

In [17]:
data['CASE_SUBMITTED'] = data['CASE_SUBMITTED'].dt.strftime('%Y-%m-%d')
data['DECISION_DATE'] = data['DECISION_DATE'].dt.strftime('%Y-%m-%d')

In [18]:
data.EMPLOYER_ADDRESS.isnull().sum()

5

In [19]:
data['EMPLOYER_ADDRESS'] = data['EMPLOYER_ADDRESS'].apply(utils.emp_address)

In [20]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

21

In [21]:
data.EMPLOYER_POSTAL_CODE = data['EMPLOYER_POSTAL_CODE'].apply(utils.update_fix_zip)

zip code error:  068574
Not a number:  T0K 0E0
Not a number:  T0K0E0
zip code error:  704098
zip code error:  289273
zip code error:  900
zip code error:  721310
zip code error:  721310
zip code error:  914223
Not a number:  nan
zip code error:  454111
zip code error:  198010
Not a number:  nan
Not a number:  NY
Not a number:  FULT0N
Not a number:  nan
zip code error:  33
zip code error:  942301
Not a number:  PAL0 ALT0
zip code error:  100017
Not a number:  NY
zip code error:  113272
Not a number:  NEW Y0RK
Not a number:  NY
Not a number:  NEW Y0RK
Not a number:  M0NTG0MERY
zip code error:  541511
zip code error:  541511
zip code error:  778435
Not a number:  V6H 4E4
zip code error:  511210
zip code error:  541511
Not a number:  nan
zip code error:  075038
Not a number:  DALLAS
zip code error:  2751927513
zip code error:  2751927513
zip code error:  750653
zip code error:  537171
zip code error:  110050
Not a number:  C00K
Not a number:  PHILADELPHIA
Not a number:  nan
zip code error:

In [22]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

167

In [23]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

32

In [24]:
data.WORKSITE_POSTAL_CODE = data['WORKSITE_POSTAL_CODE'].apply(utils.update_fix_zip)

zip code error:  300321
zip code error:  954301
Not a number:  NEW Y0RK
zip code error:  331354
zip code error:  327027
Not a number:  MD4413
zip code error:  138133
zip code error:  138133
zip code error:  111801
zip code error:  300022
Not a number:  L0S ANGELES
zip code error:  799103
zip code error:  300804
zip code error:  770521
Not a number:  L0S ANGELES
zip code error:  070004
Not a number:  10038NA
zip code error:  300936
zip code error:  7046377020
zip code error:  300936
Not a number:  NEW Y0RK
zip code error:  941407
zip code error:  068503
zip code error:  900013
zip code error:  100
zip code error:  070004
zip code error:  070004
Not a number:  NEW Y0RK
Not a number:  NEW Y0RK
zip code error:  458393
zip code error:  458393
zip code error:  7709300
zip code error:  330126
zip code error:  942301
zip code error:  196462
zip code error:  110018
zip code error:  100112
Not a number:  NEW Y0RK
zip code error:  196462
zip code error:  904050
Not a number:  nan
zip code error: 

In [25]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

434

In [26]:
data["EMPLOYER_CITY"]=data.EMPLOYER_CITY.apply(utils.uppercase_nopunct)
data["EMPLOYER_STATE"]=data.EMPLOYER_STATE.apply(utils.uppercase_nopunct)
data["WORKSITE_CITY"]=data.WORKSITE_CITY.apply(utils.uppercase_nopunct)
data["WORKSITE_STATE"]=data.WORKSITE_STATE.apply(utils.uppercase_nopunct)

In [27]:
data[data['EMPLOYER_STATE']=='PW']

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
238633,I-200-16069-297151,CERTIFIED,2016-03-10,2016-03-16,H-1B,2016-09-08,2019-09-07,"SIRI GROUP, INC.",1150 FIRST AVENUE,KING OF PRUSSIA,PW,19406,UNITED STATES OF AMERICA,NaN,6103651600,NaN,"VENKATA RAMANA, MURTHY",NEW YORK,NY,PROGRAMMER ANALYST,15-1131,COMPUTER PROGRAMMERS,541511.0,1,NaN,51730.0,Year,OES,2016.0,OFLC ONLINE DATA CENTER,52000.0,0.0,Year,N,N,EDISON,MIDDLESEX,NJ,08837,NaT
588091,I-200-16202-645185,CERTIFIED,2016-07-27,2016-08-02,H-1B,2016-08-26,2019-08-25,"TYCO ELECTRONICS CORPORATION, A TE CONNECTIVIT...",1050 WESTLAKES DRIVE,BERWYN,PW,19312,UNITED STATES OF AMERICA,NaN,71798674645,NaN,"HOGAN, ADDIE",SAN FRANCISCO,CA,STAFF SCIENTIST,19-2032,MATERIALS SCIENTISTS,334417.0,1,NaN,108971.0,Year,OES,2016.0,OFLC ONLINE DATA CENTER,121766.0,0.0,Year,N,N,MENLO PARK,SAN MATEO,CA,94025,NaT


In [28]:
data.EMPLOYER_STATE = data.EMPLOYER_STATE.replace('PW', 'PA')

In [29]:
data.EMPLOYER_STATE.isnull().sum()

34

In [30]:
data.EMPLOYER_STATE = data[['CASE_NUMBER','EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE']].apply(utils.update_fix_states, axis =1)

Old state was nan and new state is TX Case Number: I-200-16092-805772


In [31]:
data.EMPLOYER_STATE.isnull().sum()

33

In [32]:
data.WORKSITE_STATE.isnull().sum()

10

In [33]:
data.WORKSITE_STATE = data[['CASE_NUMBER','WORKSITE_STATE', 'WORKSITE_POSTAL_CODE']].apply(utils.worksite_fix_states, axis =1)

Old state was FM and new state is FL Case Number: I-200-16210-112407
Old state was nan and new state is NJ Case Number: I-200-16033-045239
Old state was MH and new state is TX Case Number: I-200-16083-246761
Old state was nan and new state is WA Case Number: I-200-16068-898401
Old state was nan and new state is CA Case Number: I-200-16076-763661
Old state was PW and new state is PA Case Number: I-200-16083-655863
Old state was MH and new state is MA Case Number: I-200-16083-197446
Old state was MH and new state is MA Case Number: I-200-16064-496099
Old state was PW and new state is PA Case Number: I-200-16055-778323


In [34]:
data.WORKSITE_STATE.isnull().sum()

7

In [54]:
data.JOB_TITLE = data['JOB_TITLE'].apply(utils.emp_address)

In [36]:
 data['DOT_CODE'] = None

In [ ]:
def update_fix_socCode(x):
    soc = str(x).strip()
    soc = re.sub('\s', '', soc)
    if(re.match('^[0-9]{2}[\.\-][0-9]{4}[\.\-][0-9]{2}$', soc)):
        return soc[:2] + '-' + soc[3:7] + '.' + soc[8:]
    if(re.match('^[0-9]{2}-[0-9]{4}$', soc)):
        return soc
    elif(re.match('^[0-9]{2}\.[0-9]{4}$', soc)):
        return re.sub('\.', '-', soc)
    elif(re.match('^[0-9]{2}\/[0-9]{4}$', soc)):
        return re.sub('\/', '-', soc)
    elif(re.match('^[0-9]{6}$', soc)):
        return soc[:2] + '-' + soc[2:]
    else:
        print('SOC code error: ', x, '|', soc)
        return soc

In [37]:
 data.SOC_CODE = data.SOC_CODE.apply(utils.update_fix_socCode)

SOC code error:  11-321.00 | 11-321.00
SOC code error:  13-111 | 13-111
SOC code error:  13-181.01 | 13-181.01
SOC code error:  13321 | 13321
SOC code error:  151031.00 | 151031.00
SOC code error:  15-111 | 15-111
SOC code error:  15-112 | 15-112
SOC code error:  15-199.02 | 15-199.02
SOC code error:  17-011 | 17-011
SOC code error:  18850 N 56TH ST | 18850N56THST
SOC code error:  69638.00 | 69638.00
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan
SOC code error:  nan | nan


In [38]:
data.SOC_CODE.unique()

array(['10-1021.00', '11-1011', '11-1021', '11-1021.00', '11-2011',
       '11-2021', '11-2021.00', '11-2022', '11-2022.00', '11-2031',
       '11-2031.00', '11-3011', '11-3021', '11-3021.00', '11-3031',
       '11-3031.00', '11-3031.01', '11-3031.02', '11-3051', '11-3051.00',
       '11-3051.01', '11-3061', '11-3061.00', '11-3071', '11-3071.01',
       '11-3071.03', '11-3111', '11-3121', '11-3131', '11-321.00',
       '11-9013', '11-9021', '11-9021.00', '11-9031', '11-9032',
       '11-9032.00', '11-9033', '11-9039', '11-9039.02', '11-9041',
       '11-9041.00', '11-9051', '11-9051.00', '11-9071', '11-9081',
       '11-9111', '11-9111.00', '11-9121', '11-9121.01', '11-9141',
       '11-9151', '11-9161', '11-9199', '11-9199.01', '11-9199.03',
       '11-9199.04', '11-9199.99', '13-1081.02', '13-1011', '13-1021',
       '13-1022', '13-1023', '13-1023.00', '13-1031', '13-1041',
       '13-1041.00', '13-1041.07', '13-1051', '13-1051.00', '13-1070.00',
       '13-1071', '13-1071.00', '13-1

In [39]:
data.TOTAL_WORKERS.isnull().sum()

0

In [40]:
data.EMPLOYER_PHONE = data.EMPLOYER_PHONE.apply(utils.emp_address)

In [41]:
data['EMPLOYER_AREA_CODE'] = [
    str(i)[:3] if len(str(i)) == 10 else np.nan for i in data['EMPLOYER_PHONE']
]

In [42]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Week', 'Bi-Weekly', 'Month', nan], dtype=object)

In [43]:
data['WAGE_UNIT_OF_PAY'] = data['WAGE_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [44]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Y', 'H', 'W', 'B', 'M', None], dtype=object)

In [45]:
data['PW_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Week', 'Bi-Weekly', 'Month', nan], dtype=object)

In [46]:
data['PW_UNIT_OF_PAY'] = data['PW_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [47]:
data['PW_UNIT_OF_PAY'].unique()

array(['Y', 'H', 'W', 'B', 'M', None], dtype=object)

In [ ]:
def calc_wage(x):
    wf = x.WAGE_RATE_OF_PAY_FROM
    wt = x.WAGE_RATE_OF_PAY_TO
    if pd.isnull(wt):
        return wf

    if wf <=wt:
        return (wf + wt)/2
    else:
        return wf

In [48]:
data['WAGE_RATE_OF_PAY_FROM'] = data['WAGE_RATE_OF_PAY_FROM'].replace(0, np.nan)
data['WAGE_RATE_OF_PAY_TO'] = data['WAGE_RATE_OF_PAY_TO'].replace(0, np.nan)
data['WAGE_RATE_OF_PAY'] = data[['WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO']].apply(utils.calc_wage, axis =1)

In [49]:
data.head(30)

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_WAGE_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE,DOT_CODE,EMPLOYER_AREA_CODE,WAGE_RATE_OF_PAY
0,I-200-12240-490687,CERTIFIED-WITHDRAWN,2012-08-27,2015-10-16,H-1B,2013-02-25,2016-02-24,UNIVERSITY OF MICHIGAN,PIERPONT COMMONS INTERNATIONAL CENTER,ANN ARBOR,MI,48109,UNITED STATES OF AMERICA,NaN,7347634081,NaN,",",NaN,NaN,POSTDOCTORAL RESEARCH FELLOW,10-1021.00,BIOCHEMISTS AND BIOPHYSICISTS,611310.0,1,NaN,36067.00,Y,OES,2012.0,NaN,41000.00,NaN,Y,N,N,ANN ARBOR,WASHTENAW,MI,48109,2012-08-31,None,734,41000.00
1,I-200-13053-847481,CERTIFIED-WITHDRAWN,2013-02-25,2016-01-13,H-1B,2013-08-24,2016-08-23,"GOODMAN NETWORKS, INC.",6400 INTERNATIONAL PARKWAY SUITE 1000,PLANO,TX,75093,UNITED STATES OF AMERICA,NaN,9724215173,NaN,"HULME, RANDALL",ADDISON,TX,CHIEF OPERATING OFFICER,11-1011,CHIEF EXECUTIVES,238210.0,1,NaN,242674.00,Y,OES,2012.0,OFLC ONLINE DATA CENTER,400000.00,NaN,Y,N,N,PLANO,COLLIN,TX,75093,2013-03-01,None,972,400000.00
2,I-200-13088-054259,CERTIFIED-WITHDRAWN,2013-04-23,2015-12-10,H-1B,2013-04-29,2016-04-28,"PORTS AMERICA GROUP, INC.",525 WASHINGTON BOULEVARD,JERSEY CITY,NJ,07310,UNITED STATES OF AMERICA,NaN,7326353899,NaN,"HENNER, JANET",NEW YORK,NY,CHIEF PROCESS OFFICER,11-1011,CHIEF EXECUTIVES,488310.0,1,NaN,193066.00,Y,OES,2012.0,OFLC ONLINE DATA CENTER,264000.00,451100.0,Y,N,N,JERSEY CITY,HUDSON,NJ,07310,2013-04-29,None,732,357550.00
3,I-200-13144-034110,CERTIFIED-WITHDRAWN,2013-05-24,2016-01-19,H-1B,2013-10-01,2016-09-30,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",1551 WEWATTA STREET,DENVER,CO,80202,UNITED STATES OF AMERICA,NaN,3037444739,NaN,"Shackelford, Andrew",PHOENIX,AZ,REGIONAL PRESIDEN AMERICAS,11-1011,CHIEF EXECUTIVES,3363.0,1,NaN,220314.00,Y,OES,2012.0,OFLC ONLINE DATA CENTER,220314.00,600000.0,Y,N,N,DENVER,DENVER,CO,80202,2013-05-29,None,303,410157.00
4,I-200-13172-415116,WITHDRAWN,2013-06-26,2016-05-20,H-1B,2013-10-18,2016-10-17,PEABODY INVESTMENTS CORP.,701 MARKET STREET,ST. LOUIS,MO,63101,UNITED STATES OF AMERICA,NaN,3145883065,NaN,"BAILEY, MICHAEL",ADDISON,TX,PRESIDENT MONGOLIA AND INDIA,11-1011,CHIEF EXECUTIVES,213113.0,1,NaN,75.73,H,OES,2012.0,OFLC ONLINE DATA CENTER,171.63,NaN,H,N,N,ST. LOUIS,ST. LOUIS,MO,63101,NaT,None,314,171.63
5,I-200-13219-306739,CERTIFIED-WITHDRAWN,2013-08-07,2015-10-30,H-1B,2013-09-15,2016-09-15,BURGER KING CORPORATION,5505 BLUE LAGOON DRIVE,MIAMI,FL,33126,UNITED STATES OF AMERICA,NaN,3053783000,NaN,"Hall, Raazia",Atlanta,GA,EXECUTIVE V P GLOBAL DEVELOPMENT AND PRESIDENT...,11-1011,CHIEF EXECUTIVES,722513.0,1,NaN,225000.00,Y,Other,2012.0,TOWERS WATSON 2012 CSR TOP MANAGEMENT COMPENSA...,225000.00,NaN,Y,N,N,MIAMI,MIAMI-DADE,FL,33126,2013-08-12,None,305,225000.00
6,I-200-13317-861241,CERTIFIED-WITHDRAWN,2013-11-20,2016-02-22,H-1B,2013-12-02,2016-12-01,BT AND MK ENERGY AND COMMODITIES,1395 BRICKELL AVENUE,MIAMI,FL,33131,UNITED STATES OF AMERICA,NaN,7864520948,NaN,"GAVICA PEREZ, DENISE",CORAL GABLES,FL,CHIEF OPERATING OFFICER,11-1011,CHIEF EXECUTIVES,213112.0,1,NaN,91021.00,Y,Other,2013.0,OFLC ONLINE DATA CENTER,120000.00,NaN,Y,N,N,HOUSTON,HOUSTON,TX,77084,2013-11-25,None,786,120000.00
7,I-200-14037-755844,CERTIFIED-WITHDRAWN,2014-02-07,2015-11-16,H-1B,2014-08-07,2017-08-06,"GLOBO MOBILE TECHNOLOGIES, INC.",1054 SOUTH DE ANZA BOULEVARD SUITE 105,SAN JOSE,CA,95129,UNITED STATES OF AMERICA,NaN,7087777930,NaN,"TAYLOR, MARC",NEW YORK,NY,CHIEF OPERATIONS OFFICER,11-1011,CHIEF EXECUTIVES,5112.0,1,NaN,15

In [50]:
#LOOK AT YO CLEAN DATA
data.WAGE_RATE_OF_PAY_FROM.isnull().sum()

10

In [51]:
#Select only the columns that your data has that are in the master list
cdata = data[['CASE_SUBMITTED', 'CASE_NUMBER', 'CASE_STATUS', 'DECISION_DATE', 'VISA_CLASS',
    'JOB_TITLE', 'DOT_CODE', 'SOC_CODE', 'SOC_NAME', 'FULL_TIME_POSITION',
    'EMPLOYER_NAME', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
    'EMPLOYER_AREA_CODE', 'NAIC_CODE', 'TOTAL_WORKERS', 
    'WORKSITE_CITY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 
    'WAGE_RATE_OF_PAY', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY']]

In [52]:
cdata.head()

,CASE_SUBMITTED,CASE_NUMBER,CASE_STATUS,DECISION_DATE,VISA_CLASS,JOB_TITLE,DOT_CODE,SOC_CODE,SOC_NAME,FULL_TIME_POSITION,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_AREA_CODE,NAIC_CODE,TOTAL_WORKERS,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,2012-08-27,I-200-12240-490687,CERTIFIED-WITHDRAWN,2015-10-16,H-1B,POSTDOCTORAL RESEARCH FELLOW,None,10-1021.00,BIOCHEMISTS AND BIOPHYSICISTS,NaN,UNIVERSITY OF MICHIGAN,PIERPONT COMMONS INTERNATIONAL CENTER,ANN ARBOR,MI,48109,734,611310.0,1,ANN ARBOR,MI,48109,41000.00,Y,36067.00,Y
1,2013-02-25,I-200-13053-847481,CERTIFIED-WITHDRAWN,2016-01-13,H-1B,CHIEF OPERATING OFFICER,None,11-1011,CHIEF EXECUTIVES,NaN,"GOODMAN NETWORKS, INC.",6400 INTERNATIONAL PARKWAY SUITE 1000,PLANO,TX,75093,972,238210.0,1,PLANO,TX,75093,400000.00,Y,242674.00,Y
2,2013-04-23,I-200-13088-054259,CERTIFIED-WITHDRAWN,2015-12-10,H-1B,CHIEF PROCESS OFFICER,None,11-1011,CHIEF EXECUTIVES,NaN,"PORTS AMERICA GROUP, INC.",525 WASHINGTON BOULEVARD,JERSEY CITY,NJ,07310,732,488310.0,1,JERSEY CITY,NJ,07310,357550.00,Y,193066.00,Y
3,2013-05-24,I-200-13144-034110,CERTIFIED-WITHDRAWN,2016-01-19,H-1B,REGIONAL PRESIDEN AMERICAS,None,11-1011,CHIEF EXECUTIVES,NaN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",1551 WEWATTA STREET,DENVER,CO,80202,303,3363.0,1,DENVER,CO,80202,410157.00,Y,220314.00,Y
4,2013-06-26,I-200-13172-415116,WITHDRAWN,2016-05-20,H-1B,PRESIDENT MONGOLIA AND INDIA,None,11-1011,CHIEF EXECUTIVES,NaN,PEABODY INVESTMENTS CORP.,701 MARKET STREET,ST. LOUIS,MO,63101,314,213113.0,1,ST. LOUIS,MO,63101,171.63,H,75.73,H


In [53]:
#Write out your clean data to a file

if not os.path.exists('cleaned_data'):
        os.mkdir('cleaned_data')
        
new_filename="2016.csv"
cdata.to_csv("cleaned_data/{}".format(new_filename), encoding='utf-8')